In [ ]:

# Get cifar100
from helpers import *

batch_size_100 = 512
workers_100 = 6
cpu_prefetch_100 = 10
gpu_prefetch_100 = 5

batch_size_100_val = 128
workers_100_val = 2
cpu_prefetch_100_val = 2
gpu_prefetch_100_val = 2

cifar100_train, cifar100_val = get_cifar(is_cifar_10 = False, recompute=False, redownload=False)
cifar_100_train_loader = gen_data_loader(data = cifar100_train, batch_size=batch_size_100, workers=workers_100-workers_100_val, cpu_prefetch=cpu_prefetch_100-cpu_prefetch_100_val, gpu_prefetch=gpu_prefetch_100-gpu_prefetch_100_val)
print("Val")
cifar_100_val_loader = gen_data_loader(data = cifar100_val, batch_size=batch_size_100_val, workers=workers_100_val, cpu_prefetch=cpu_prefetch_100_val, gpu_prefetch=gpu_prefetch_100_val)
cifar_100_deletables.extend([cifar100_train, cifar100_val, cifar_100_train_loader, cifar_100_val_loader])


In [ ]:
retrain_vgg_100 = True
if retrain_vgg_100:
    try:
        delete_deletables([vgg_100])
    except:
        pass
    vgg_100 = VggNet(
        in_chan = 3,
        in_dim = (32, 32),
        n_classes = 100,
        block_params=[
            (2, ConvParams(
                kernel=3,
                out_chan=64,
                stride=1,
                padding='same',
            )),
            (2,
             ConvParams(
                 kernel=3,
                 out_chan=128,
                 stride=1,
                 padding='same',
             )),
            (2,
             ConvParams(
                 kernel=3,
                 out_chan=256,
                 stride=1,
                 padding='same',
             )),
            (3,
             ConvParams(
                 kernel=3,
                 out_chan=512,
                 stride=1,
                 padding='same',
             )),
            (3,
             ConvParams(
                 kernel=3,
                 out_chan=512,
                 stride=1,
                 padding='same',
             )),
        ],
        fc_params=[
            4096,
            4096,
        ],
    )
    vgg_100.to('cuda')
    vgg_100.train_model(
        epochs = 50,
        train_loader = cifar_100_train_loader,
        val_loader = cifar_100_val_loader,
        optimizer = torch.optim.Adam,
        optimizer_kwargs={'lr': 1e-4},
        print_epoch=1,
        header_epoch=10,
        min_accuracy=0.5,
    )
    torch.save(vgg_100, 'models/vgg_100.pth')
else:
    vgg_100 = torch.load('models/vgg_100.pth')
    vgg_100.to('cuda')
